In [1]:
# import modules
import uproot, sys, time, math, pickle, os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import awkward as ak
from tqdm import tqdm
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from matplotlib.ticker import FormatStrFormatter
import matplotlib.ticker as ticker
from scipy.special import betainc
from scipy.stats import norm

# import config functions
from jet_faking_plot_config import getWeight, zbi, sample_dict, getVarDict
from plot_var import variables, variables_data, ntuple_names, ntuple_names_BDT
from n_1_iteration_functions import get_best_cut, calculate_significance, apply_cut_to_fb, apply_all_cuts, compute_total_significance, n_minus_1_optimizer
# from cut_config import cut_config

# Set up plot defaults
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = 14.0,10.0  # Roughly 11 cm wde by 8 cm high  
mpl.rcParams['font.size'] = 20.0 # Use 14 point font
sns.set(style="whitegrid")

font_size = {
    "xlabel": 17,
    "ylabel": 17,
    "xticks": 15,
    "yticks": 15,
    "legend": 14
}

plt.rcParams.update({
    "axes.labelsize": font_size["xlabel"],  # X and Y axis labels
    "xtick.labelsize": font_size["xticks"],  # X ticks
    "ytick.labelsize": font_size["yticks"],  # Y ticks
    "legend.fontsize": font_size["legend"]  # Legend
})

tot = []
data = pd.DataFrame()
unweighted_bcut, weighted_bcut, unweighted_acut, weighted_acut = [], [], [], []
ntuple_names = ['ggHyyd','Zjets','Zgamma','Wgamma','Wjets','gammajet_direct', 'data23']

def test(fb):
    # checking if there are any none values
    mask = ak.is_none(fb['met_tst_et'])
    n_none = ak.sum(mask)
    print("Number of none values: ", n_none)
    # if n_none > 0:
    #     fb = fb[~mask]
    # print("Events after removing none values: ", len(fb), ak.sum(ak.is_none(fb['met_tst_et'])))

def print_cut(ntuple_name, fb, label):
    print(f"Unweighted Events {label}: ", len(fb))
    if ntuple_name == 'data23':
        print(f"Weighted Events {label}: ", sum(getWeight(fb, ntuple_name, jet_faking=True)))
    else: 
        print(f"Weighted Events {label}: ", sum(getWeight(fb, ntuple_name)))

for i in range(len(ntuple_names)):
    ucut, wcut = [], []
    start_time = time.time()
    ntuple_name = ntuple_names[i]
    if ntuple_name == 'data23': # data
        path = f"/data/fpiazza/ggHyyd/Ntuples/MC23d/withVertexBDT/data23_y_BDT_score.root" 
        print('processing file: ', path)
        f = uproot.open(path)['nominal']
        fb = f.arrays(variables_data, library="ak")
        fb['VertexBDTScore'] = fb['BDTScore'] # renaming BDTScore to ensure this is recognized as Vertex BDT Score
        
        fb = fb[ak.num(fb['ph_eta']) > 0]     # for abs(ak.firsts(fb['ph_eta'])) to have value to the reweighting
                
        mask1 = (ak.firsts(fb['ph_topoetcone40'])-2450.)/ak.firsts(fb['ph_pt']) > 0.1   # jet_faking_photon cut
        fb = fb[mask1]
        fb = fb[fb['n_ph_baseline'] == 1]

    else: # MC
        path = f"/data/tmathew/ntups/mc23d/{ntuple_name}_y.root" 
        path_BDT = f"/data/fpiazza/ggHyyd/Ntuples/MC23d/withVertexBDT/mc23d_{ntuple_name}_y_BDT_score.root" 
        print('processing file: ', path)
        f = uproot.open(path)['nominal']
        fb = f.arrays(variables, library="ak")

        # add BDT score to fb
        f_BDT = uproot.open(path_BDT)['nominal']
        fb_BDT = f_BDT.arrays(["event", "BDTScore"], library="ak")
        tmp = fb["event"] == fb_BDT["event"]
        if np.all(tmp) == True:
            fb["VertexBDTScore"] = fb_BDT["BDTScore"]
        else: 
            print("Something is wrong, need arranging")

        fb = fb[ak.num(fb['ph_eta']) > 0]     # for abs(ak.firsts(fb['ph_eta'])) to have value to the reweighting
        fb = fb[fb['n_ph'] == 1]
        
        # Zjets and Wjets (rule out everything except for e->gamma)
        if ntuple_name == 'Zjets' or ntuple_name == 'Wjets':
            mask = ak.firsts(fb['ph_truth_type']) == 2
            fb = fb[mask]
        
        # goodPV on signal only
        if ntuple_name == 'ggHyyd':
            fb = fb[ak.num(fb['pv_z']) > 0]
            good_pv_tmp = (np.abs(ak.firsts(fb['pv_truth_z']) - ak.firsts(fb['pv_z'])) <= 0.5)
            fb = fb[good_pv_tmp]

    print_cut(ntuple_name, fb, 'before cut')
    wcut.append(sum(getWeight(fb, ntuple_name)))

    fb = fb[fb['n_mu_baseline'] == 0]
    wcut.append(sum(getWeight(fb, ntuple_name)))
    fb = fb[fb['n_el_baseline'] == 0]
    wcut.append(sum(getWeight(fb, ntuple_name)))
    fb = fb[fb['n_tau_baseline'] == 0]
    wcut.append(sum(getWeight(fb, ntuple_name)))
    fb = fb[fb['trigger_HLT_g50_tight_xe40_cell_xe70_pfopufit_80mTAC_L1eEM26M']==1]
    wcut.append(sum(getWeight(fb, ntuple_name)))
    fb = fb[ak.num(fb['ph_pt']) > 0] # prevent none values in Tbranch
    fb = fb[ak.firsts(fb['ph_pt']) >= 50000] # ph_pt cut (basic cut)
    wcut.append(sum(getWeight(fb, ntuple_name)))
    fb = fb[fb['met_tst_et'] >= 100000] # MET cut (basic cut)
    wcut.append(sum(getWeight(fb, ntuple_name)))
    fb = fb[fb['n_jet_central'] <= 4] # n_jet_central cut (basic cut)
    wcut.append(sum(getWeight(fb, ntuple_name)))

    mt_tmp = np.sqrt(2 * fb['met_tst_et'] * ak.firsts(fb['ph_pt']) * 
                            (1 - np.cos(fb['met_tst_phi'] - ak.firsts(fb['ph_phi'])))) / 1000
    mask1 = mt_tmp >= 100 # trigger cut
    fb = fb[mask1]
    wcut.append(sum(getWeight(fb, ntuple_name)))

    print_cut(ntuple_name, fb, 'after basic cut')


    ucut.append(len(fb))

    unweighted_acut.append(ucut)
    weighted_acut.append(wcut)
    test(fb) # check for none value

    print(f"Reading Time for {ntuple_name}: {(time.time()-start_time)} seconds\n")



    tot.append(fb)

    fb = 0
    fb_BDT = 0
    tmp = 0


processing file:  /data/tmathew/ntups/mc23d/ggHyyd_y.root
Unweighted Events before cut:  86910
Weighted Events before cut:  8732.987955115426
Unweighted Events after basic cut:  4587
Weighted Events after basic cut:  461.9221957176054
Number of none values:  0
Reading Time for ggHyyd: 7.100672721862793 seconds

processing file:  /data/tmathew/ntups/mc23d/Zjets_y.root
Unweighted Events before cut:  3242488
Weighted Events before cut:  676616.903247458
Unweighted Events after basic cut:  9208
Weighted Events after basic cut:  706.2264841437278
Number of none values:  0
Reading Time for Zjets: 168.94695448875427 seconds

processing file:  /data/tmathew/ntups/mc23d/Zgamma_y.root
Unweighted Events before cut:  3423357
Weighted Events before cut:  249851.55031619867
Unweighted Events after basic cut:  1715357
Weighted Events after basic cut:  52667.25081568077
Number of none values:  0
Reading Time for Zgamma: 143.32745480537415 seconds

processing file:  /data/tmathew/ntups/mc23d/Wgamma_y.r

In [2]:
def getCutDict():
    cut_dict = {}
signal_name = 'ggHyyd'  # Define signal dataset
cut_name = 'basic'

def getCutDict():
    cut_dict = {}
    # Reduced Features
    cut_dict['VertexBDTScore'] = {
        'lowercut': np.arange(0, 0.3+0.01, 0.01) # VertexBDTScore > cut
    }
    cut_dict['balance'] = {
        'lowercut': np.arange(0, 1.5 + 0.01, 0.01), # balance > cut
        'uppercut': np.arange(1.5, 9, 0.05) # balance < cut
    }
    cut_dict['dmet'] = {
        'lowercut': np.arange(-30000, 10000 + 100, 100), # dmet > cut
        'uppercut': np.arange(10000, 100000 + 100, 100), # -10000 < dmet < cut
    }
    cut_dict['metsig'] = {
        'lowercut': np.arange(0, 10 + 1, 1), # metsig > cut
        'uppercut': np.arange(10, 30 + 1, 1), # metsig < cut 
    }
    cut_dict['jetterm'] = {
        'lowercut': np.arange(0, 150000+500, 500) # jetterm > cut
    }
    cut_dict['dphi_met_phterm'] = {
        'lowercut': np.arange(1, 2 + 0.01, 0.01), # dphi_met_phterm > cut
    }
    cut_dict['dphi_met_central_jet'] = {
        'lowercut': np.arange(1.5, 2.8, 0.01)
    }
    cut_dict['ph_eta'] = {
        'uppercut': np.arange(1, 2.5 + 0.01, 0.01), # ph_eta < cut
    }
    cut_dict['ph_pt'] = {
        'lowercut': np.arange(50000, 100000 + 1000, 1000),  # ph_pt > cut
        'uppercut': np.arange(100000, 300000 + 1000, 1000),  # ph_pt > cut
    }

    # Other Features
    cut_dict['dphi_jj'] = {
        'uppercut': np.arange(1, 3.14 + 0.01, 0.01) # dphi_jj < cut
    }
    cut_dict['dphi_phterm_jetterm'] = {
        'lowercut': np.arange(1, 2.5 + 0.05, 0.05), # dphi_phterm_jetterm > cut
        'uppercut': np.arange(2, 4 + 0.1, 0.1) # dphi_phterm_jetterm < cut
    }
    cut_dict['jet_central_eta'] = {
        'lowercut': np.arange(-2.5, 0+0.01, 0.01), # jet_central_eta > cut
        'uppercut': np.arange(0, 2.5+0.01, 0.01) # jet_central_eta < cut
    }
    cut_dict['jet_central_pt2'] = {
        'lowercut': np.arange(20000, 100000+1000, 1000) # jet_central_pt2 > cut
    }
    cut_dict['metsigres'] = {
        'lowercut': np.arange(8600, 15000, 100),
        'uppercut': np.arange(12000, 60000, 100)
    }
    cut_dict['met_noJVT'] = {
        'lowercut': np.arange(50000, 120000, 100),
        'uppercut': np.arange(100000, 250000, 100)
    }
    cut_dict['softerm'] = {
        'uppercut': np.arange(10000, 40000, 100)
    }
    cut_dict['n_jet_central'] = {
        'uppercut': np.arange(0, 8+1, 1) # njet < cut
    }

    return cut_dict
cut_config = getCutDict()

In [ ]:
%%time
signal_name='ggHyyd'
initial_cut = []
tot2 = tot

# < -- Initial Cut on all variables (maximize the significance * acceptance) -- > 
for cut_var, cut_types in cut_config.items():
    for cut_type, cut_values in cut_types.items():
        sig_simple_list, sigacc_simple_list, acceptance_values = calculate_significance(
            cut_var, cut_type, cut_values, tot2, ntuple_names, signal_name, getVarDict, getWeight
        )

        best_cut, best_sig, idx = get_best_cut(cut_values, sigacc_simple_list) 
        
        if idx == 0 or idx == len(sigacc_simple_list) - 1: # I chose to use index to indicate not to make unnecessary cut (for initial cut)
            print(cut_var, idx, len(sigacc_simple_list))
            continue
            
        result = {
            "cut_var": cut_var,
            "cut_type": cut_type,
            "best_cut": best_cut,
            "best_sig_x_acc": best_sig,
            "significance": sig_simple_list[idx],
            "acceptance": acceptance_values[idx]
        }

        print(result)
        initial_cut.append(dict(list(result.items())[:3]))

{'cut_var': 'VertexBDTScore', 'cut_type': 'lowercut', 'best_cut': 0.09, 'best_sig_x_acc': 0.5300952216935515, 'significance': 0.6397698893038751, 'acceptance': 82.85716951611163}


In [ ]:
tot2_initial_cut = apply_all_cuts(tot2, ntuple_names, initial_cut, getVarDict)
final_significance = compute_total_significance(tot2_initial_cut, ntuple_names, signal_name, getVarDict, getWeight)
print('after initial cutting, signficance: ', final_significance)

In [7]:
%%time
# < -- n-1 iterations until no further improvement (max significance) -- >
optimized_cuts, final_significance = n_minus_1_optimizer(
    initial_cut, cut_config, tot2, ntuple_names, signal_name, getVarDict, getWeight, final_significance
)
print('after optimized cutting, signficance: ', final_significance)



--- Iteration 1 ---
Updating dphi_jj (uppercut): 2.7600000000000016 → 1.5500000000000005  (sig 0.79 → 0.82)
Updating dphi_phterm_jetterm (lowercut): 1.4500000000000004 → 1.6500000000000006  (sig 0.82 → 0.86)
Updating dphi_phterm_jetterm (uppercut): 3.000000000000001 → 2.8000000000000007  (sig 0.86 → 0.89)
Updating metsigres (lowercut): 10300 → 10700  (sig 0.89 → 0.89)
Updating metsigres (uppercut): 34200 → 20100  (sig 0.89 → 1.09)
Updating met_noJVT (lowercut): 100000 → 103000  (sig 1.09 → 1.13)
Updating met_noJVT (uppercut): 249100 → 248300  (sig 1.13 → 1.13)

--- Iteration 2 ---
Updating dphi_jj (uppercut): 1.5500000000000005 → 1.5800000000000005  (sig 1.13 → 1.13)
Updating dphi_phterm_jetterm (lowercut): 1.6500000000000006 → 1.8000000000000007  (sig 1.13 → 1.15)
Updating metsigres (lowercut): 10700 → 11500  (sig 1.15 → 1.17)
Updating metsigres (uppercut): 20100 → 19300  (sig 1.17 → 1.17)
Updating met_noJVT (lowercut): 103000 → 103400  (sig 1.17 → 1.18)
Updating met_noJVT (uppercut)

In [11]:
print( ' < -- Final Optimized Cuts -- > ')
# print(optimized_cuts)

for cut in optimized_cuts:
    var = cut['cut_var']
    val = cut['best_cut']
    if cut['cut_type'] == 'uppercut':
        print(f"{var} <= {val}")
    elif cut['cut_type'] == 'lowercut':
        print(f"{var} >= {val}")
        
print('after optimized cutting, signficance: ', final_significance)


 < -- Final Optimized Cuts -- > 
dphi_jj <= 1.5600000000000005
dphi_phterm_jetterm >= 1.8000000000000007
dphi_phterm_jetterm <= 2.8000000000000007
metsigres >= 11500
metsigres <= 19200
met_noJVT >= 103000
met_noJVT <= 243100
n_jet_central <= 4
after optimized cutting, signficance:  1.176777241481177


In [13]:
tot2_optimized_cuts = apply_all_cuts(tot2, ntuple_names, optimized_cuts, getVarDict)

In [14]:
# < -- Save data after cuts to a csv file for BDT input -- >
Vars = [
    'balance', 
    'VertexBDTScore',
    'dmet',
    'dphi_jj',
    'dphi_met_central_jet',
    'dphi_met_phterm',
    'dphi_met_ph',
    'dphi_met_jetterm',
    'dphi_phterm_jetterm',
    'dphi_ph_centraljet1',
    'ph_pt',
    'ph_eta',
    'ph_phi',
    'jet_central_eta',
    'jet_central_pt1',
    'jet_central_pt2',
    'jetterm',
    'jetterm_sumet',
    'metsig',
    'metsigres',
    'met',
    'met_noJVT',
    'metplusph',
    'failJVT_jet_pt1',
    'softerm',
    'n_jet_central'
]

data_list = []

for j in range(len(ntuple_names)):
    process = ntuple_names[j]
    fb = tot2_optimized_cuts[j] 
    
    data_dict = {}
    
    for var in Vars:
        var_config = getVarDict(fb, process, var_name=var)
        data_dict[var] = var_config[var]['var']
    
    weights = getWeight(fb, process)
    data_dict['weights'] = weights
    
    n_events = len(weights)
    data_dict['process'] = [process] * n_events
    label = 1 if process == 'ggHyyd' else 0
    data_dict['label'] = [label] * n_events
    
    df_temp = pd.DataFrame(data_dict)
    data_list.append(df_temp)

df_all = pd.concat(data_list, ignore_index=True)
df_all.head()

df_all.to_csv("/data/jlai/ntups/csv/jet_faking_BDT_input_basic_reduced2.csv", index=False)